In [1]:
import os
from PIL import Image
import matplotlib.pyplot as plt
import pickle

In [2]:
import colorthief

In [3]:
from colorthief import ColorThief

In [4]:
from scipy.spatial import KDTree

In [5]:
import webcolors

In [6]:
from webcolors import (
    CSS3_HEX_TO_NAMES,
#     css3_hex_to_names,
    hex_to_rgb,
)

In [7]:
with open('css_color_to_name.p','rb') as F:
    color_to_name = pickle.load(F)

In [8]:
restricted_css3 = {}
for key in CSS3_HEX_TO_NAMES.keys():
    if color_to_name[key] != 'none':
        restricted_css3[key] = CSS3_HEX_TO_NAMES[key]

In [9]:
CSS3_HEX_TO_NAMES = restricted_css3

In [10]:
image_folder = 'static/dataset/coco_val_set/'

In [11]:
image_files = os.listdir(image_folder)
image_paths = [os.path.join(image_folder, i) for i in image_files]

In [12]:
# CSS3_HEX_TO_NAMES

In [13]:
experiment_color_list = ['red','pink','orange','yellow','purple','green','blue','brown','gray','black','white']

In [14]:
import numpy as np
from math import sqrt

In [15]:
def get_similarity(c1, c2):
    """Computes the pearson correlation coefficient for two colors. The result
    will be between 1.0 (very similar) and -1.0 (no similarity)."""
#     c1 = color_to_rgb(color1)
#     c2 = color_to_rgb(color2)
    s1 = sum(c1)
    s2 = sum(c2)
    sp1 = sum(map(lambda c: pow(c, 2), c1))
    sp2 = sum(map(lambda c: pow(c, 2), c2))
    sp = sum(map(lambda x: x[0] * x[1], zip(c1, c2)))

    return (sp - (s1 * s2 / 3.0)) / sqrt((sp1 - pow(s1, 2) / 3.0) * (sp2 - pow(s2, 2) / 3.0))

In [16]:
from scipy.stats.stats import pearsonr

In [17]:
def convert_using_pearson(rgb_tuple):
    best_similarity = 0
    best_color = 'NA'
    for color in CSS3_HEX_TO_NAMES.keys():
        color_rgb = hex_to_rgb(color)
        similarity_score = pearsonr(color_rgb,rgb_tuple)[0]
        
        if similarity_score > best_similarity:
            best_similarity = similarity_score
            best_color = CSS3_HEX_TO_NAMES[color]
            best_rgb = color_rgb
    return best_color, [best_rgb[0],best_rgb[1],best_rgb[2]]

def convert_rgb_to_names(rgb_tuple):
    
    # a dictionary of all the hex and their respective names in css3
    css3_db = CSS3_HEX_TO_NAMES
    names = []
    rgb_values = []
    for color_hex, color_name in css3_db.items():
        names.append(color_name)
        rgb_values.append(hex_to_rgb(color_hex))
    
    kdt_db = KDTree(rgb_values)
    distance, index = kdt_db.query(rgb_tuple)
    return names[index], [rgb_values[index][0],rgb_values[index][1],rgb_values[index][2]]


def display_matched_colors(original_rgb,matched_rgb):
    f, axarr = plt.subplots(1,2)
    axarr[0].imshow([[original_rgb]])
    axarr[1].imshow([[matched_rgb]])
    plt.show()

In [18]:
inverted_css3 = {}
for key,value in CSS3_HEX_TO_NAMES.items():
    inverted_css3[value] = key

In [19]:
histogram_dominant_color = {}

In [20]:
from tqdm import tqdm

In [21]:
# dominant_color

In [22]:
for impath in tqdm(image_paths):
    color_thief = ColorThief(impath)
    dominant_color = color_thief.get_color(quality=1)
    named_color,_ = convert_rgb_to_names(dominant_color)
    easy_color_name = color_to_name[inverted_css3[named_color]]
    histogram_dominant_color[impath] = easy_color_name
    break

  0%|          | 0/5000 [00:00<?, ?it/s]


In [23]:
named_color,_ = convert_rgb_to_names(dominant_color)
easy_color_name = color_to_name[inverted_css3[named_color]]

(162, 134, 78)

In [24]:
# with open('histogram_dominant_color.p','wb') as F:
#     pickle.dump(histogram_dominant_color,F)